In [9]:
install.packages('cmprsk',lib='Rlib')
install.packages("survivalROC",lib='Rlib')

In [99]:
library(cmprsk,lib='Rlib')
library(data.table)
library(dplyr)
library(survivalROC,lib='Rlib')

In [88]:
# load primary data
vte <- read.csv("data/xs_VTE_main_processed.csv", header=TRUE)
logical_columns <- sapply(vte, function(x) all(x %in% c("True", "False")))
vte <- vte %>%
  mutate_at(vars(names(logical_columns)[logical_columns]), ~ ifelse(. == "True", TRUE, FALSE))
vte$CAT_DEATH_ENDPT[vte$CAT_DEATH_ENDPT=='False'] <- 0
vte$CAT_DEATH_ENDPT[vte$CAT_DEATH_ENDPT=='True'] <- 1
vte$CAT_DEATH_ENDPT[vte$CAT_DEATH_ENDPT=='2'] <- 2
                          
vte2 <- read.csv("data/vte2_main.csv", header=TRUE)
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='False'] <- 0
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='True'] <- 1
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='2'] <- 2
logical_columns <- sapply(vte2, function(x) all(x %in% c("True", "False")))
vte2 <- vte2 %>%
  mutate_at(vars(names(logical_columns)[logical_columns]), ~ ifelse(. == "True", TRUE, FALSE))

resbio_VTE <- read.csv('data/resbio_VTE_processed.csv', header=TRUE)
resbio_VTE$X.ctDNA<-resbio_VTE$X.ctDNA=='True'
sydney <- read.csv('data/sydney_processed.csv', header=TRUE)
sydney$X.ctDNA<-sydney$X.ctDNA=='True'
resbio_all <- rbindlist(list(resbio_VTE, sydney),fill=TRUE)

In [3]:
xs <- read.table('data/data_mutations_extended_xs_1_6_23.txt', 
                 header = TRUE, sep = "\t", comment.char = "#")
vc = table(xs$Hugo_Symbol)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"EOF within quoted string"
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"number of items read is not a multiple of the number of columns"


In [56]:
cois <- list("Non.Small.Cell.Lung.Cancer", "Breast.Cancer", "Pancreatic.Cancer",
            "Melanoma", "Prostate.Cancer", "Bladder.Cancer",
            "Esophagogastric.Cancer", "Hepatobiliary.Cancer", "Colorectal.Cancer")

columns = c("exp(coef)","exp(-coef)","2.5%","97.5%") 
df = data.frame(matrix(nrow = 0, ncol = length(columns)))
colnames(df) = columns

for (x in names(sort(vc,decreasing=TRUE))) {
    if (x %in% colnames(vte)){
        if(sum(vte[,x])>=30){
            print(sum(vte[,x]))
           # z<-crr(vte$stop,vte$CAT_DEATH_ENDPT,vte[c(x,unlist(cois))])
           # df[x,]<-(summary(z)$conf.int)[x,]
        }
    }
}
print(df)
write.csv(df, "Routputs/geneStrat_ctDNA_CAT.csv", row.names=TRUE)

[1] 1272
[1] 465
[1] 339
[1] 254
[1] 148
[1] 187
[1] 82
[1] 192
[1] 92
[1] 80
[1] 76
[1] 63
[1] 107
[1] 121
[1] 91
[1] 83
[1] 64
[1] 105
[1] 121
[1] 75
[1] 76
[1] 80
[1] 70
[1] 69
[1] 65
[1] 38
[1] 34
[1] 41
[1] exp(coef)  exp(-coef) 2.5%       97.5%     
<0 rows> (or 0-length row.names)


In [12]:
cois <- list("Non.Small.Cell.Lung.Cancer", "Breast.Cancer", "Pancreatic.Cancer",
            "Melanoma", "Prostate.Cancer", "Bladder.Cancer",
            "Esophagogastric.Cancer", "Hepatobiliary.Cancer", "Colorectal.Cancer")

columns = c("exp(coef)","exp(-coef)","2.5%","97.5%") 
df = data.frame(matrix(nrow = 0, ncol = length(columns)))
colnames(df) = columns

for (x in cois) {
    z<-crr(vte$stop,vte$CAT_DEATH_ENDPT,vte[,x])
    df[x,]<-summary(z)$conf.int
}
print(df)
write.csv(df, "Routputs/cancerStrat_CAT.csv", row.names=TRUE)

                           exp(coef) exp(-coef)       2.5%     97.5%
Non.Small.Cell.Lung.Cancer 2.0780516  0.4812200 1.73271150 2.4922202
Breast.Cancer              0.4604712  2.1716886 0.32204834 0.6583910
Pancreatic.Cancer          1.5729802  0.6357359 1.19994881 2.0619768
Melanoma                   0.1930692  5.1794912 0.09161161 0.4068884
Prostate.Cancer            0.7191252  1.3905785 0.45796398 1.1292176
Bladder.Cancer             1.2749223  0.7843615 0.87094816 1.8662728
Esophagogastric.Cancer     0.7652417  1.3067767 0.43933103 1.3329239
Hepatobiliary.Cancer       1.3649216  0.7326428 0.86494959 2.1538954
Colorectal.Cancer          0.3765680  2.6555631 0.15774170 0.8989598


In [5]:
cois <- list("Non.Small.Cell.Lung.Cancer", "Breast.Cancer", "Pancreatic.Cancer",
            "Melanoma", "Prostate.Cancer", "Bladder.Cancer",
            "Esophagogastric.Cancer", "Hepatobiliary.Cancer", "Colorectal.Cancer")

columns = c("exp(coef)","exp(-coef)","2.5%","97.5%") 
df = data.frame(matrix(nrow = 0, ncol = length(columns)))
colnames(df) = columns

for (x in cois) {
    temp<-vte[c(unlist(vte[x])),]
    z<-crr(temp$stop,temp$CAT_DEATH_ENDPT,temp$log10.cfDNA.concentration.)
    df[x,]<-summary(z)$conf.int
}
temp<-vte[apply(vte[c(unlist(cois))], 1, function(row) all(row == FALSE)), ]
z<-crr(temp$stop,temp$CAT_DEATH_ENDPT,temp$log10.cfDNA.concentration.)
df['Other',]<-summary(z)$conf.int
print(df)
write.csv(df, "Routputs/cancerStrat_cfDNA_CAT.csv", row.names=TRUE)

                           exp(coef) exp(-coef)      2.5%     97.5%
Non.Small.Cell.Lung.Cancer  2.365530  0.4227382 1.8621735  3.004947
Breast.Cancer               1.839788  0.5435409 1.0898203  3.105852
Pancreatic.Cancer           1.858706  0.5380086 1.1696450  2.953708
Melanoma                    8.569877  0.1166878 1.8976683 38.701596
Prostate.Cancer             1.772679  0.5641180 0.9817868  3.200685
Bladder.Cancer              1.356662  0.7371031 0.7022942  2.620743
Esophagogastric.Cancer      2.715552  0.3682493 0.5543826 13.301685
Hepatobiliary.Cancer        1.710310  0.5846895 0.4917487  5.948483
Colorectal.Cancer           3.296229  0.3033770 1.5799255  6.876986
Other                       2.713992  0.3684609 1.7065948  4.316053


In [3]:
cois <- list("Non.Small.Cell.Lung.Cancer", "Breast.Cancer", "Pancreatic.Cancer",
            "Melanoma", "Prostate.Cancer", "Bladder.Cancer",
            "Esophagogastric.Cancer", "Hepatobiliary.Cancer", "Colorectal.Cancer")

columns = c("exp(coef)","exp(-coef)","2.5%","97.5%") 
df = data.frame(matrix(nrow = 0, ncol = length(columns)))
colnames(df) = columns

for (x in cois) {
    temp<-vte[c(unlist(vte[x])),]
    z<-crr(temp$stop,temp$CAT_DEATH_ENDPT,temp$X.ctDNA)
    df[x,]<-summary(z)$conf.int
}
temp<-vte[apply(vte[c(unlist(cois))], 1, function(row) all(row == FALSE)), ]
z<-crr(temp$stop,temp$CAT_DEATH_ENDPT,temp$X.ctDNA)
df['Other',]<-summary(z)$conf.int
print(df)
write.csv(df, "Routputs/cancerStrat_ctDNA_CAT.csv", row.names=TRUE)

                            exp(coef) exp(-coef)      2.5%      97.5%
Non.Small.Cell.Lung.Cancer  1.8399863 0.54348229 1.3304195   2.544723
Breast.Cancer               2.8180203 0.35485905 0.9845286   8.066031
Pancreatic.Cancer           2.2197911 0.45049283 1.2907088   3.817649
Melanoma                   18.5270017 0.05397527 2.2283682 154.036390
Prostate.Cancer             3.4856744 0.28688853 0.7988208  15.209827
Bladder.Cancer              1.0730132 0.93195499 0.4774411   2.411517
Esophagogastric.Cancer      6.8785527 0.14537942 0.8769164  53.955528
Hepatobiliary.Cancer        0.9143161 1.09371368 0.3358440   2.489173
Colorectal.Cancer           0.8924634 1.12049408 0.1539538   5.173570
Other                       3.2389868 0.30873852 1.6569848   6.331401


In [15]:
print(z<-crr(resbio_all[resbio_all$stop>180,]$stop,
             resbio_all[resbio_all$stop>180,]$CAT_DEATH_ENDPT,
             resbio_all[resbio_all$stop>180,]$X.ctDNA))
summary(z)$conf.int

convergence:  TRUE 
coefficients:
resbio_all[resbio_all$stop > 180, ]$X.ctDNA1 
                                      0.4832 
standard errors:
[1] 0.3461
two-sided p-values:
resbio_all[resbio_all$stop > 180, ]$X.ctDNA1 
                                        0.16 


,exp(coef),exp(-coef),2.5%,97.5%
"resbio_all[resbio_all$stop > 180, ]$X.ctDNA1",1.621246,0.6168097,0.8227671,3.194631


In [14]:
print(z<-crr(vte[vte$stop>180,]$stop,
             vte[vte$stop>180,]$CAT_DEATH_ENDPT,
             vte[vte$stop>180,]$X.ctDNA))
summary(z)$conf.int

convergence:  TRUE 
coefficients:
vte[vte$stop > 180, ]$X.ctDNA1 
                        0.4718 
standard errors:
[1] 0.1716
two-sided p-values:
vte[vte$stop > 180, ]$X.ctDNA1 
                         0.006 


,exp(coef),exp(-coef),2.5%,97.5%
"vte[vte$stop > 180, ]$X.ctDNA1",1.602888,0.6238739,1.145074,2.243742


In [6]:
print(z<-crr(vte$stop,vte$CAT_DEATH_ENDPT,vte$X.ctDNA))
summary(z)$conf.int

convergence:  TRUE 
coefficients:
vte$X.ctDNA1 
      0.9105 
standard errors:
[1] 0.1136
two-sided p-values:
vte$X.ctDNA1 
     1.1e-15 


,exp(coef),exp(-coef),2.5%,97.5%
vte$X.ctDNA1,2.485543,0.4023265,1.98942,3.105391


In [97]:
mtv <- read.csv("data/MTV_128_mrn.csv")
resbio_mtv <- merge(resbio_all[,c("MRN","X.ctDNA","KHORANA.SCORE",'stop','CAT_DEATH_ENDPT',
                                 "log10.cfDNA.concentration.","chemotherapy")], 
                    new_dataframe, by = "MRN", all.x = FALSE, all.y = FALSE)
xs_mtv<-merge(vte[,c("MRN","X.ctDNA","KHORANA.SCORE",'stop','CAT_DEATH_ENDPT',
                                 "log10.cfDNA.concentration.","chemotherapy")], 
                    new_dataframe, by = "MRN", all.x = FALSE, all.y = FALSE)

all_mtv<-rbind(resbio_mtv,xs_mtv)
all_mtv$chemotherapy<-all_mtv$chemotherapy=="True"
all_mtv$log10.MTV<-log10(all_mtv$MTV+1)
all_mtv$CAT_DEATH_ENDPT = as.numeric(all_mtv$CAT_DEATH_ENDPT)

In [98]:
z<-crr(all_mtv$stop,
       all_mtv$CAT_DEATH_ENDPT,
       all_mtv[,c("X.ctDNA","log10.MTV",
                  "KHORANA.SCORE","log10.cfDNA.concentration.","chemotherapy")])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/multivar_MTV.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,3.993218,0.2504246,1.1865931,13.438297
log10.MTV,1.197784,0.8348751,0.5420369,2.646843
KHORANA.SCORE,1.186951,0.8424948,0.6684478,2.107648
log10.cfDNA.concentration.,0.998440,1.0015624,0.3105980,3.209558
chemotherapy,3.201308,0.3123723,1.4330750,7.151318


In [20]:
z<-crr(vte$stop,vte$CAT_DEATH_ENDPT,vte[c("X.ctDNA",
                  "KHORANA.SCORE","N.organ.sites","log10.cfDNA.concentration.","chemotherapy")])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/validation_ctDNA_CAT.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,1.655933,0.6038890,1.299020,2.110910
KHORANA.SCORE,1.133262,0.8824081,1.033132,1.243098
N.organ.sites,1.121745,0.8914684,1.065183,1.181310
log10.cfDNA.concentration.,1.614733,0.6192973,1.358886,1.918751
chemotherapy,1.705112,0.5864716,1.404400,2.070214


In [33]:
z<-crr(vte[!vte$STAGE_IV_DX=='True',]$stop,
       vte[!vte$STAGE_IV_DX=='True',]$CAT_DEATH_ENDPT,
       vte[!vte$STAGE_IV_DX=='True',][c("X.ctDNA",
                  "KHORANA.SCORE","log10.cfDNA.concentration.",'chemotherapy')])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/multivar_ctDNA_CAT_notstageIV.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,1.865177,0.5361421,1.3006572,2.674713
KHORANA.SCORE,1.139589,0.8775092,0.9752799,1.331580
log10.cfDNA.concentration.,2.138877,0.4675352,1.6179903,2.827454
chemotherapy,1.925479,0.5193514,1.3699967,2.706188


In [31]:
z<-crr(vte[vte$STAGE_IV_DX=='True',]$stop,
       vte[vte$STAGE_IV_DX=='True',]$CAT_DEATH_ENDPT,
       vte[vte$STAGE_IV_DX=='True',][c("X.ctDNA",
                  "KHORANA.SCORE","N.organ.sites","log10.cfDNA.concentration.",'chemotherapy')])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/multivar_ctDNA_CAT_stageIV.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,1.439574,0.6946500,1.0629785,1.949590
KHORANA.SCORE,1.080209,0.9257470,0.9579238,1.218104
N.organ.sites,1.034782,0.9663875,0.9677962,1.106403
log10.cfDNA.concentration.,1.360047,0.7352687,1.0997704,1.681922
chemotherapy,1.410453,0.7089923,1.1192115,1.777480


In [26]:
z<-crr(vte[!vte$chemotherapy,]$stop,
       vte[!vte$chemotherapy,]$CAT_DEATH_ENDPT,
       vte[!vte$chemotherapy,][c("X.ctDNA",
                  "KHORANA.SCORE","N.organ.sites","log10.cfDNA.concentration.")])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/multivar_ctDNA_CAT_nochemotherapy.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,1.942192,0.5148821,1.371180,2.750996
KHORANA.SCORE,1.188194,0.8416133,1.034595,1.364596
N.organ.sites,1.182935,0.8453549,1.101739,1.270115
log10.cfDNA.concentration.,1.781300,0.5613879,1.382113,2.295781


In [27]:
z<-crr(vte[vte$chemotherapy,]$stop,
       vte[vte$chemotherapy,]$CAT_DEATH_ENDPT,
       vte[vte$chemotherapy,][c("X.ctDNA",
                  "KHORANA.SCORE","N.organ.sites","log10.cfDNA.concentration.")])
summary(z)$conf.int
write.csv(summary(z)$conf.int, "Routputs/multivar_ctDNA_CAT_chemotherapy.csv", row.names=TRUE)

,exp(coef),exp(-coef),2.5%,97.5%
X.ctDNA,1.350136,0.7406659,0.9760670,1.867565
KHORANA.SCORE,1.049095,0.9532022,0.9217658,1.194014
N.organ.sites,1.052448,0.9501658,0.9742074,1.136972
log10.cfDNA.concentration.,1.468634,0.6809047,1.1674659,1.847495


In [8]:
columns = c("exp(coef)","exp(-coef)","2.5%","97.5%") 
df = data.frame(matrix(nrow = 0, ncol = length(columns)))
colnames(df) = columns

z<-crr(vte2$stop,vte2$CAT_DEATH_ENDPT,vte2$X.ctDNA)
df<-rbind(df,summary(z)$conf.int)

z<-crr(resbio_all$stop,resbio_all$CAT_DEATH_ENDPT,resbio_all$X.ctDNA)
df<-rbind(df,summary(z)$conf.int)

z<-crr(resbio_VTE$stop,resbio_VTE$CAT_DEATH_ENDPT,resbio_VTE$X.ctDNA)
df<-rbind(df,summary(z)$conf.int)

z<-crr(sydney$stop,sydney$CAT_DEATH_ENDPT,sydney$X.ctDNA)
df<-rbind(df,summary(z)$conf.int)

print(df)
write.csv(df, "Routputs/validation_ctDNA_CAT.csv", row.names=TRUE)

Warning message in regularize.values(x, y, ties, missing(ties), na.rm = na.rm):
"collapsing to unique 'x' values"
Warning message in regularize.values(x, y, ties, missing(ties), na.rm = na.rm):
"collapsing to unique 'x' values"


                    exp(coef) exp(-coef)     2.5%    97.5%
vte2$X.ctDNA1        2.487571  0.4019985 1.601484 3.863923
resbio_all$X.ctDNA1  2.302976  0.4342208 1.433551 3.699692
resbio_VTE$X.ctDNA1  2.279461  0.4387002 1.298909 4.000237
sydney$X.ctDNA1      2.472615  0.4044301 1.031239 5.928622


In [104]:
cois <- list("Non.Small.Cell.Lung.Cancer", "Breast.Cancer", "Pancreatic.Cancer",
            "Melanoma", "Prostate.Cancer", "Bladder.Cancer",
            "Esophagogastric.Cancer", "Hepatobiliary.Cancer", "Colorectal.Cancer")
temp<-vte[c(unlist(vte$X.ctDNA)),]
print(z<-crr(temp$stop,temp$CAT_DEATH_ENDPT,
             temp[c("previous.aspirin",
                   "lt_start","AGE",unlist(cois))]))
summary(z)$conf.int

convergence:  TRUE 
coefficients:
          previous.aspirin                   lt_start 
                -1.099e-01                 -9.738e-05 
                       AGE Non.Small.Cell.Lung.Cancer 
                -9.461e-06                  6.806e-01 
             Breast.Cancer          Pancreatic.Cancer 
                -2.182e-01                  7.953e-01 
                  Melanoma            Prostate.Cancer 
                -1.388e-01                  2.306e-01 
            Bladder.Cancer     Esophagogastric.Cancer 
                 4.318e-01                  2.209e-01 
      Hepatobiliary.Cancer          Colorectal.Cancer 
                 3.791e-01                 -8.943e-01 
standard errors:
 [1] 1.370e-01 5.032e-05 3.570e-03 2.224e-01 2.729e-01 2.611e-01 4.532e-01
 [8] 3.275e-01 3.077e-01 3.548e-01 3.327e-01 7.217e-01
two-sided p-values:
          previous.aspirin                   lt_start 
                    0.4200                     0.0530 
                       AGE No

,exp(coef),exp(-coef),2.5%,97.5%
previous.aspirin,0.8959211,1.1161697,0.68495211,1.171870
lt_start,0.9999026,1.0000974,0.99980401,1.000001
AGE,0.9999905,1.0000095,0.99301794,1.007012
Non.Small.Cell.Lung.Cancer,1.9751296,0.5062959,1.27725510,3.054313
Breast.Cancer,0.8039293,1.2438904,0.47089611,1.372495
Pancreatic.Cancer,2.2151241,0.4514420,1.32791713,3.695091
Melanoma,0.8704042,1.1488915,0.35807745,2.115753
Prostate.Cancer,1.2593183,0.7940804,0.66272215,2.392983
Bladder.Cancer,1.5400020,0.6493498,0.84249636,2.814975
Esophagogastric.Cancer,1.2472554,0.8017604,0.62218422,2.500298


# RSF risk scores for dynamic AUC

In [6]:
vte2 <- read.csv("data/vte2_main_wriskscores.csv", header=TRUE)
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='False'] <- 0
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='True'] <- 1
vte2$CAT_DEATH_ENDPT[vte2$CAT_DEATH_ENDPT=='2'] <- 2
logical_columns <- sapply(vte2, function(x) all(x %in% c("True", "False")))
vte2 <- vte2 %>%
  mutate_at(vars(names(logical_columns)[logical_columns]), ~ ifelse(. == "True", TRUE, FALSE))

In [17]:
for (c in list("Khorana.Score","LB.","All")){

    ROC.1 = survivalROC(Stime = vte2$stop, status = vte2$CAT_DEATH_ENDPT==1, 
                        marker = vte2[c], predict.time = 180, lambda = 0.05)

    write.csv(ROC.1, file = paste('rsf_scores/vte2_',c,'_dROC.csv'))
}